# Testing environment for DoSE

## Setup

### Load libraries

In [1]:
import pandas as pd
import numpy as np 
import gseapy
from biothings_client import get_client

### Define data

In [2]:
seeds_file = "Input/0007079.txt"
betweenness_file = "Input/0007079_added_200_dmd_betweenness_hub_0.01.txt"
significance_file = "Input/0007079_added_200_dmd_significance_hub_1.txt"
diseases_file = "Input/ICD10_commROCG_raw.txt"
disease_clusters_file = "Input/ICD10_commROCG_cluster.txt"

### Load data

In [3]:
seeds = pd.read_csv(seeds_file, sep="\t", header=None)[0]
betweenness = pd.read_csv(betweenness_file, sep="\t")['node']
significance = pd.read_csv(significance_file, sep="\t")['node']
diseases = pd.read_csv(diseases_file, sep="\t", header=None)
disease_clusters = pd.read_csv(disease_clusters_file, sep="\t", header=None)

In [4]:
import timeit
start = timeit.default_timer()

stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  4.1903999999703956e-05


## Mapper

In [5]:
id_type_key = {'entrez':'entrezgene','ensembl':'ensembl.gene','symbol':'symbol','uniprot':'uniprot.Swiss-Prot','mondo':'mondo'}
gene_ids=['uniprot.Swiss-Prot','symbol','ensembl.gene','entrezgene']

In [29]:
def preprocess_results(mapping, multicol, singlecol, key, explode=False):
    
    def convert_to_string(cell, key):
        if str(cell) != 'nan':
            extracted_ids = [val.get(key) for val in cell]
            return ';'.join(str(e) for e in list(set(extracted_ids)))
        return cell
    mapping[multicol] = mapping[multicol].apply(lambda x: convert_to_string(x, key)) if multicol in mapping else np.nan
    if singlecol in mapping:
        mapping[multicol].fillna(mapping[singlecol], inplace=True)
        mapping = mapping.drop(columns=[singlecol])
    if explode:
        mapping = mapping[multicol].split(';').explode(multicol)
        mapping.rename(columns={multicol: singlecol}, inplace = True)
    return mapping


def get_prev_mapping(in_set, id_type, file, sep):
    # ===== Get mapping from local mapping file =====
    mapping = pd.read_csv(file, sep=sep, header=0, dtype=str)
    if id_type == "ICD-10":
        mapping = split_and_expand_column(data=mapping, split_string=",", column_name="ICD-10")
    # ==== Map given disease set ====
    id_type = id_type_key[id_type] if id_type in id_type_key else id_type
    mapped_set = mapping[mapping[id_type].isin(in_set)]
    # ===== Get missing values =====
    missing = list(set(in_set) - set(mapping[id_type]))
    return mapped_set, missing, mapping
    

def get_gene_mapping(gene_set, id_type):
    """
    Simple converter.

    :param gene_set: Set of gene ids
    :return: Dataframe
    """
    # ===== Get mapping from previous mappings =====
    df, missing, prev_mapping = get_prev_mapping(in_set=gene_set, id_type=id_type, file='gene_id_mapping.csv', sep=",")
    # ===== Get mapping for missing values =====
    if len(missing) > 0:
        mg = get_client("gene")
        mapping = mg.querymany(missing, scopes=id_type_key[id_type], fields=','.join(gene_ids),
                     species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping = mapping.drop(columns=[id_type_key[id_type]])
        mapping.rename(columns={'query': id_type_key[id_type]}, inplace = True)
        # ===== Split if there are multiple ensembl ids =====
        if 'ensembl' in mapping:
            mapping = preprocess_results(mapping=mapping, multicol='ensembl', singlecol='ensembl.gene', key='gene', explode=True)
        mapping = mapping.drop(columns=['_id','_score'])
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('gene_id_mapping.csv', index=False)
        df = pd.concat([df, mapping]).reset_index(drop=True)
    return df

def get_gene_to_attributes(gene_set, id_type):
    """
    Simple converter.

    :param gene_set: Set of gene ids
    :return: Dataframe
    """
    # ===== Get gene ID mappings =====
    gene_mapping, _, _ = get_prev_mapping(in_set=gene_set, id_type=id_type, file='gene_id_mapping.csv', sep=",")
    df, missing, prev_mapping = get_prev_mapping(in_set=set(gene_mapping['entrezgene']), id_type='entrez', file='gene_att_mapping.csv', sep=",")
    if len(missing) > 0:
        mg = get_client("gene")
        gene_ids=['uniprot.Swiss-Prot','symbol','ensembl.gene','entrezgene']
        mapping = mg.querymany(missing, scopes=','.join(gene_ids),
                            fields='pathway.kegg.id, go.BP.id, go.CC.id, go.MF.id',
                            species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping.rename(columns={'query': 'entrezgene'}, inplace = True)
        for column in ['go.BP','go.CC','go.MF','pathway.kegg']:
            mapping = preprocess_results(mapping=mapping, multicol=column, singlecol=column+'.id', key='id')
        mapping = mapping.drop(columns=['_id','_score'])
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('gene_att_mapping.csv', index=False)        
        df = pd.concat([df, mapping]).reset_index(drop=True)
    # work with not unique values...
    mapping_subset = gene_mapping[['entrezgene', id_type_key[id_type]]].drop_duplicates()
    df = pd.merge(mapping_subset, df, on = ['entrezgene'], how = 'outer')
    df = df.drop(columns=['entrezgene'])
    df = df.fillna('').groupby([id_type_key[id_type]], as_index=False).agg({'go.BP': combine_rows, 'go.CC': combine_rows,
                                                                            'go.MF': combine_rows, 'pathway.kegg': combine_rows})
    return df


def combine_rows(x):
    return set(filter(None,';'.join(x).split(';')))

In [8]:
start = timeit.default_timer()
reference_mapping = get_gene_mapping(seeds, 'uniprot')
target_mapping = get_gene_mapping(significance, 'uniprot')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.0206533390000061


In [9]:
reference_mapping

,entrezgene,ensembl.gene,symbol,uniprot.Swiss-Prot
191,125,ENSG00000196616,ADH1B,P00325
192,2555,ENSG00000151834,GABRA2,P47869
193,126,ENSG00000248144,ADH1C,P00326
194,3356,ENSG00000102468,HTR2A,P28223


In [10]:
target_mapping

,entrezgene,ensembl.gene,symbol,uniprot.Swiss-Prot
0,1394,ENSG00000120088,CRHR1,P34998
1,1394,ENSG00000276191,CRHR1,P34998
2,104909134,ENSG00000263715,LINC02210-CRHR1,P34998
3,104909134,ENSG00000278232,LINC02210-CRHR1,P34998
4,104909134,ENSG00000282456,LINC02210-CRHR1,P34998
...,...,...,...,...
212,1742,ENSG00000132535,DLG4,P78352
213,4684,ENSG00000149294,NCAM1,P13591
214,2778,ENSG00000087460,GNAS,O95467
215,2776,ENSG00000156052,GNAQ,P50148


In [11]:
start = timeit.default_timer()
reference_kegg_mapping = get_gene_to_attributes(seeds, 'uniprot')
target_kegg_mapping = get_gene_to_attributes(significance, 'uniprot')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.05625762200000395


In [12]:
reference_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,P00325,"{GO:0042572, GO:0001523, GO:0042573, GO:0006069}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004024, GO:0008270}","{hsa00620, hsa00830, hsa00071, hsa00350, hsa00..."
1,P00326,"{GO:0042572, GO:0042573, GO:0006069}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004022, GO:0004024, GO:0008270}","{hsa00620, hsa00830, hsa00071, hsa00350, hsa00..."
2,P28223,"{GO:0006874, GO:0046718, GO:0030431, GO:000821...","{GO:0043025, GO:0030425, GO:0005886, GO:000588...","{GO:0044877, GO:0030594, GO:0051378, GO:000196...","{hsa04726, hsa04020, hsa04080, hsa04540, hsa04..."
3,P47869,"{GO:0060078, GO:1904862, GO:0051932, GO:000726...","{GO:0098982, GO:0043025, GO:0032590, GO:000588...","{GO:0030594, GO:0004890, GO:0005254, GO:002285...","{hsa05032, hsa04727, hsa04080, hsa04723, hsa04..."


In [13]:
target_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,O00459,"{GO:0015031, GO:0048010, GO:0042307, GO:004685...","{GO:0005634, GO:0005829, GO:0005942}","{GO:0001784, GO:0046982, GO:0030971, GO:001990...","{hsa04072, hsa04662, hsa05169, hsa04722, hsa05..."
1,O14492,"{GO:0050873, GO:0001922, GO:0007596, GO:005085...","{GO:0001725, GO:0005886, GO:0005737, GO:000588...","{GO:0005515, GO:0005068, GO:0042169, GO:004280...","{hsa04910, hsa04722}"
2,O14610,"{GO:0007602, GO:0007186}",{GO:0005834},"{GO:0003924, GO:0031681}","{hsa05163, hsa05032, hsa04726, hsa04727, hsa04..."
3,O14775,"{GO:0043547, GO:1901386, GO:0007186, GO:000645...","{GO:0098793, GO:0005737, GO:0005634, GO:190277...","{GO:0003924, GO:0031682, GO:0030159, GO:000551...","{hsa05163, hsa05032, hsa04726, hsa04727, hsa04..."
4,O14842,"{GO:0042593, GO:0032024, GO:0051928, GO:003007...","{GO:0005887, GO:0005886}","{GO:0008289, GO:0045125, GO:0004930}",{hsa04911}
...,...,...,...,...,...
195,Q9UN70,"{GO:0016339, GO:0050808, GO:0007156, GO:0007155}","{GO:0016020, GO:0005887}",{GO:0005509},{}
196,Q9UNN8,"{GO:0007596, GO:0050819}","{GO:0005886, GO:0005615, GO:0005887, GO:000557...","{GO:0038023, GO:0005515}",{hsa04610}
197,Q9UQC2,"{GO:0048015, GO:0007169, GO:0043306, GO:000741...","{GO:0005737, GO:0005886, GO:0005829}","{GO:0043325, GO:0005068, GO:0005547, GO:0005515}","{hsa04072, hsa04071, hsa04014, hsa04666, hsa04..."
198,Q9Y2G0,"{GO:0046854, GO:0072659}","{GO:0015629, GO:0005886, GO:0005829}",{GO:0005515},{}


In [31]:
full_ids_mapping = pd.read_csv("../new_disorders.map", sep="\t", dtype=str)
full_ids_mapping

,mondo,omim,snomedct,umls,orpha,mesh,ncit,doid,meddra,medgen,ICD-10
0,0008118,164330,716180009,C1834013,2724,C537740,NaN,NaN,NaN,NaN,NaN
1,0010439,300829,NaN,C1853577,NaN,C543241,NaN,NaN,NaN,NaN,NaN
2,0008117,164310,763829004,C1834014,98897,C563508,NaN,NaN,NaN,NaN,"G71,G71.0"
3,0009448,242600,84121007,C0268654,42062,C536285,NaN,NaN,NaN,NaN,"E72,E72.0"
4,0008119,164400,715748006,C0752120,98755,NaN,C129982,0050954,NaN,NaN,"G11,G11.8"
...,...,...,...,...,...,...,...,...,...,...,...
24115,0009507,245550,732961003,C1855551,1296,C538396,NaN,NaN,NaN,NaN,"Q87,Q87.8"
24116,0009508,245552,NaN,C1855550,NaN,C537549,NaN,NaN,NaN,NaN,NaN
24117,0009501,245340,766715000,C1855577,171690,C565449,NaN,NaN,NaN,NaN,"G72,G72.8"
24118,0009502,245348,NaN,C1855565,79244,C565448,NaN,NaN,NaN,NaN,"E74,E74.4"


In [32]:
full_ids_mapping.count()

mondo       24120
omim         8841
snomedct     8962
umls        16234
orpha        9363
mesh         8075
ncit         6953
doid         8944
meddra       1144
medgen          1
ICD-10       9561
dtype: int64

In [7]:
def split_and_expand_column(data, split_string, column_name):
    s = data[column_name].str.split(split_string, expand=True).stack()
    i = s.index.get_level_values(0)
    df2 = data.loc[i].copy()
    df2[column_name] = s.values
    return df2

def get_disease_mapping(disease_set, id_type):
    # ==== Get Mondo IDs ====
    disease_id_set,_,_ = get_prev_mapping(in_set=disease_set, id_type=id_type, file="../disorders.map", sep="\t")
    mondo_set = list(set('MONDO:'+disease_id_set['mondo']))
    # ===== Get mapping from previous mappings =====
    df, missing, prev_mapping = get_prev_mapping(in_set=mondo_set, id_type='mondo', file='disease_disgenet_mapping.csv', sep=",")
    # ==== Get disgenet values ====
    if len(missing) > 0:
        md = get_client("disease")
        mapping = md.getdiseases(missing,
                                 fields='disgenet.genes_related_to_disease.gene_id,disgenet.variants_related_to_disease.rsid,ctd.pathway_related_to_disease.kegg_pathway_id',
                                 species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping.rename(columns={'query': 'mondo'}, inplace = True)
        # transform dataframe to combine single and multiple results
        mapping = preprocess_results(mapping=mapping, multicol='disgenet.genes_related_to_disease', 
                                     singlecol='disgenet.genes_related_to_disease.gene_id', key='gene_id')
        mapping = preprocess_results(mapping=mapping, multicol='disgenet.variants_related_to_disease', 
                                     singlecol='disgenet.variants_related_to_disease.rsid', key='rsid')
        mapping = preprocess_results(mapping=mapping, multicol='ctd.pathway_related_to_disease', 
                                     singlecol='ctd.pathway_related_to_disease.kegg_pathway_id', key='kegg_pathway_id')
        mapping = mapping.drop(columns=['_id','_version','disgenet._license']) 
        #  work with nan float values
        mapping = mapping.fillna('')
        mapping = mapping.astype(str)
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('disease_disgenet_mapping.csv', index=False)
        df = pd.concat([df, mapping]).reset_index(drop=True)
    # ==== Map back to previous ids ====
    df["mondo"] = df["mondo"].str.replace("MONDO:", "")
    # work with not unique values...
    mapping_subset = disease_id_set[['mondo', id_type]].drop_duplicates()
    df = pd.merge(mapping_subset, df, on = ['mondo'], how = 'outer')
    df = df.drop(columns=['mondo'])
    df = df.fillna('').groupby(id_type, as_index = False).agg({'disgenet.genes_related_to_disease': combine_rows, 'disgenet.variants_related_to_disease': combine_rows, 'ctd.pathway_related_to_disease': combine_rows})
    return df

In [103]:
diseases[0]

0     E10
1     E11
2     E12
3     E13
4     E14
5     E66
6     F00
7     F01
8     F02
9     F03
10    G20
11    G30
12    G43
13    I10
14    I11
15    I12
16    I13
17    I15
18    I21
19    I22
20    I50
21    I63
22    I64
23    I70
24    J45
Name: 0, dtype: object

In [33]:
start = timeit.default_timer()
df = get_disease_mapping(disease_set=diseases[0], id_type='ICD-10')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.18340003999992405


<ipython-input-7-b4c792f17657>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mondo"] = df["mondo"].str.replace("MONDO:", "")


In [34]:
df

,ICD-10,disgenet.genes_related_to_disease,disgenet.variants_related_to_disease,ctd.pathway_related_to_disease
0,E10,"{387082, 3406, 54949, 723961, 1493, 3403, 3630...","{rs121908261, rs137853239, rs137853236, rs1159...","{hsa04659, hsa04072, hsa04115, hsa05169, hsa05..."
1,E11,"{1564, 84164, 1543, 64135, 2132, 2697, 3814, 4...","{rs17238540, rs7080536, rs1033656351, rs180113...","{hsa04659, hsa04072, hsa04115, hsa04662, hsa05..."
2,E13,"{3952, 57538, 84919, 2688, 1965, 6833, 3479, 8...","{rs121913143, rs121913147, rs869025178, rs8690...","{hsa04072, hsa05169, hsa05160, hsa04066, hsa04..."
3,F01,"{2260, 3586, 1401, 1742, 1398, 338, 1950, 2604...","{rs1801133, rs3813034, rs699, rs944050, rs8930...",{}
4,G20,"{3586, 6315, 27040, 26503, 1401, 51540, 23400,...","{rs12678719, rs7133914, rs1056737920, rs106465...","{hsa04659, hsa04072, hsa03018, hsa05169, hsa_M..."
5,G30,"{10312, 1564, 1719, 84164, 642, 2697, 1974, 84...","{rs147906088, rs690705, rs63750579, rs1786140,...","{hsa04659, hsa04072, hsa04115, hsa03018, hsa04..."
6,G43,{},{},"{hsa04659, hsa04726, hsa04020, hsa05220, hsa05..."
7,I10,"{1719, 1543, 64135, 2132, 2697, 3993, 9619, 11...","{rs10948071, rs11739136, rs1801133, rs11977526...","{hsa04659, hsa04072, hsa04115, hsa04662, hsa05..."
8,I11,"{1585, 101054525, 4318, 400550, 4879, 183, 919...",{},{}
9,I12,"{1634, 633, 240, 3605, 336, 4353, 2255, 81, 18...",{rs11739136},"{hsa04659, hsa04726, hsa04510, hsa04670, hsa05..."


# do the comparisson now

### set to set

In [35]:
reference_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,P00325,"{GO:0042572, GO:0001523, GO:0042573, GO:0006069}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004024, GO:0008270}","{hsa00620, hsa00830, hsa00071, hsa00350, hsa00..."
1,P00326,"{GO:0042572, GO:0042573, GO:0006069}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004022, GO:0004024, GO:0008270}","{hsa00620, hsa00830, hsa00071, hsa00350, hsa00..."
2,P28223,"{GO:0006874, GO:0046718, GO:0030431, GO:000821...","{GO:0043025, GO:0030425, GO:0005886, GO:000588...","{GO:0044877, GO:0030594, GO:0051378, GO:000196...","{hsa04726, hsa04020, hsa04080, hsa04540, hsa04..."
3,P47869,"{GO:0060078, GO:1904862, GO:0051932, GO:000726...","{GO:0098982, GO:0043025, GO:0032590, GO:000588...","{GO:0030594, GO:0004890, GO:0005254, GO:002285...","{hsa05032, hsa04727, hsa04080, hsa04723, hsa04..."


In [36]:
target_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,O00459,"{GO:0015031, GO:0048010, GO:0042307, GO:004685...","{GO:0005634, GO:0005829, GO:0005942}","{GO:0001784, GO:0046982, GO:0030971, GO:001990...","{hsa04072, hsa04662, hsa05169, hsa04722, hsa05..."
1,O14492,"{GO:0050873, GO:0001922, GO:0007596, GO:005085...","{GO:0001725, GO:0005886, GO:0005737, GO:000588...","{GO:0005515, GO:0005068, GO:0042169, GO:004280...","{hsa04910, hsa04722}"
2,O14610,"{GO:0007602, GO:0007186}",{GO:0005834},"{GO:0003924, GO:0031681}","{hsa05163, hsa05032, hsa04726, hsa04727, hsa04..."
3,O14775,"{GO:0043547, GO:1901386, GO:0007186, GO:000645...","{GO:0098793, GO:0005737, GO:0005634, GO:190277...","{GO:0003924, GO:0031682, GO:0030159, GO:000551...","{hsa05163, hsa05032, hsa04726, hsa04727, hsa04..."
4,O14842,"{GO:0042593, GO:0032024, GO:0051928, GO:003007...","{GO:0005887, GO:0005886}","{GO:0008289, GO:0045125, GO:0004930}",{hsa04911}
...,...,...,...,...,...
195,Q9UN70,"{GO:0016339, GO:0050808, GO:0007156, GO:0007155}","{GO:0016020, GO:0005887}",{GO:0005509},{}
196,Q9UNN8,"{GO:0007596, GO:0050819}","{GO:0005886, GO:0005615, GO:0005887, GO:000557...","{GO:0038023, GO:0005515}",{hsa04610}
197,Q9UQC2,"{GO:0048015, GO:0007169, GO:0043306, GO:000741...","{GO:0005737, GO:0005886, GO:0005829}","{GO:0043325, GO:0005068, GO:0005547, GO:0005515}","{hsa04072, hsa04071, hsa04014, hsa04666, hsa04..."
198,Q9Y2G0,"{GO:0046854, GO:0072659}","{GO:0015629, GO:0005886, GO:0005829}",{GO:0005515},{}


In [37]:
reference_dict = dict()
for att_type in reference_kegg_mapping.columns[1:]:
    if att_type not in reference_dict:
        reference_dict[att_type] = set.union(*reference_kegg_mapping[att_type])
reference_dict

{'go.BP': {'GO:0001505',
  'GO:0001523',
  'GO:0001659',
  'GO:0006069',
  'GO:0006836',
  'GO:0006874',
  'GO:0007165',
  'GO:0007186',
  'GO:0007187',
  'GO:0007200',
  'GO:0007202',
  'GO:0007208',
  'GO:0007210',
  'GO:0007214',
  'GO:0007268',
  'GO:0007568',
  'GO:0007613',
  'GO:0008219',
  'GO:0008284',
  'GO:0010513',
  'GO:0014059',
  'GO:0014065',
  'GO:0014824',
  'GO:0014832',
  'GO:0030431',
  'GO:0034220',
  'GO:0042391',
  'GO:0042493',
  'GO:0042572',
  'GO:0042573',
  'GO:0043267',
  'GO:0043406',
  'GO:0044380',
  'GO:0045600',
  'GO:0045821',
  'GO:0045907',
  'GO:0046718',
  'GO:0048148',
  'GO:0050731',
  'GO:0050877',
  'GO:0050965',
  'GO:0050966',
  'GO:0051209',
  'GO:0051932',
  'GO:0051967',
  'GO:0060078',
  'GO:0070374',
  'GO:0098664',
  'GO:1902476',
  'GO:1904862',
  'GO:2000300'},
 'go.CC': {'GO:0005654',
  'GO:0005829',
  'GO:0005886',
  'GO:0005887',
  'GO:0005901',
  'GO:0030285',
  'GO:0030424',
  'GO:0030425',
  'GO:0031410',
  'GO:0032590',
  'GO